# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

In [0]:
df_22 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/uefa_2022.csv")
df_21 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/uefa_2021.csv")
df_20 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/carlosalonsomingo@gmail.com/uefa_2020.csv")

### Requerimientos:  
-Puedes utilizar SQL o PySpark

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
from pyspark.sql import functions as F 

df1_TEAM_HOME_WITH_MOST_GOALS = df_20.groupBy(
    "TEAM_NAME_HOME",
).agg(
    F.sum(F.col("TEAM_HOME_SCORE")).alias("TEAM_HOME_SCORE")
).orderBy(F.desc("TEAM_HOME_SCORE")).limit(3)
df1_TEAM_HOME_WITH_MOST_GOALS.display()

TEAM_NAME_HOME,TEAM_HOME_SCORE
Manchester City,15.0
Real Madrid,14.0
Bayern Munich,13.0


2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [0]:
#Vamos ha hacer primero el calculo de cuantos han ganado la posesion cuando es local

df_poss_local = (df_21.groupBy("TEAM_NAME_HOME")
                 .agg(F.sum(
                     (F.split(F.col("POSSESSION_HOME"), '%')[0].cast("int") > 50).cast("int")
                 ).alias("count_poss_greater_than_50"))
                )

# df_poss_local.display()
#luego vamos a calcularlo para el visitante

df_poss_visitante = (df_21.groupBy("TEAM_NAME_AWAY")
                 .agg(F.sum(
                     (F.split(F.col("POSSESSION_AWAY"), '%')[0].cast("int") > 50).cast("int")
                 ).alias("count_poss_greater_than_50"))
                )


#Unimos los los dataframes
combined_df = df_poss_local.union(df_poss_visitante)
#Agrupamos sumando
TEAM_WITH_MAJORITY_POSSESSION_df = (combined_df.groupBy("TEAM_NAME_HOME")
               .agg(F.sum(F.col("count_poss_greater_than_50")).alias("GAME_COUNT"))
               .orderBy(F.col("GAME_COUNT").desc())).limit(1)
TEAM_WITH_MAJORITY_POSSESSION_df.display()


TEAM_NAME_HOME,GAME_COUNT
Liverpool,9


3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
# df_22.display()

#Vamos a agregar un columna para saber quien ha perdido cada partido
df1 = df_22.withColumn("looser",
                       F.when(F.col("TEAM_HOME_SCORE") > F.col("TEAM_AWAY_SCORE"),
                              "away")
                       .otherwise(
                         F.when(F.col("TEAM_HOME_SCORE") < F.col("TEAM_AWAY_SCORE"),
                                "home").otherwise("any")
                       ))

#nos olvidamos de los any(ya que han empatado)

df1 = df1.filter(
  F.col("looser") != "any"
)
# Ahora que tenemos el looser de cada partido obtendremos los que han ganado mas enfrentamientos y aun asi han perdido el partido
df1 = df1.select( "*",
  (F.when(
    (F.col("looser") == "away"),
    (F.split(F.col("DUELS_WON_AWAY"), "%")[0]).cast("int") > 50)
  .otherwise(
    (F.split(F.col("DUELS_WON_AWAY"), "%")[0]).cast("int") < 50)).alias("cumple")
)

#Ahora tenemos una columna donde sabemos  los que cumplen la condicion, deberemos obtener los que son true para completar la consulta
df1 = df1.filter(
  F.col("cumple") == True
)
# #Ya sabemos los que si cumplen, ahora debemos agrupar por stage y después convertir los equipos en una lista 

TEAM_WON_DUEL_LOST_GAME_STAGE_WISE_df = df1.groupBy(
  "STAGE",
  "looser"
).agg(
  F.when(
    F.col("looser") == "away",
    F.collect_list(F.col("TEAM_NAME_AWAY")))
  .otherwise(F.collect_list(F.col("TEAM_NAME_HOME")))
  ).drop(F.col("looser"))


TEAM_WON_DUEL_LOST_GAME_STAGE_WISE_df.display()



STAGE,CASE WHEN (looser = away) THEN collect_list(TEAM_NAME_AWAY) ELSE collect_list(TEAM_NAME_HOME) END
Group stage: Matchday 5,"List(Real Madrid, K�benhavn, Juventus, Maccabi Haifa, Rangers, Viktoria Plze?, Marseille)"
Group stage: Matchday 6,"List(Napoli, Atletico Madrid, Internazionale, Salzburg, Dinamo Zagreb)"
Group stage: Matchday 3,"List(Rangers, Atletico Madrid, Bayer Leverkusen, Viktoria Plze?, Barcelona, Sporting CP, Dinamo Zagreb, AC Milan, Celtic, K�benhavn, Maccabi Haifa)"
Group stage: Matchday 1,"List(Chelsea, K�benhavn, Juventus, Maccabi Haifa, Rangers, Liverpool, Porto, Bayer Leverkusen, Viktoria Plze?, Marseille)"
Group stage: Matchday 6,"List(Celtic, Sevilla)"
Semi-finals second leg,"List(Real Madrid, Milan)"
Quarter-finals first leg,"List(Napoli, Chelsea)"
Round of 16 second leg,"List(Paris Saint-Germain, Dortmund, Eintracht Frankfurt, Liverpool, Leipzig)"
Group stage: Matchday 4,"List(Juventus, Ajax, Eintracht Frankfurt)"
Round of 16 first leg,"List(Chelsea, Tottenham, FC Porto)"
